In [195]:
import pandas as pd
pd.set_option('display.max_rows', 50)

## 1. Data Understanding

### 1.1 Data Loading

In [ ]:
nih = pd.read_parquet('../data_analysis/nih_analysis.parquet', engine='fastparquet')
nsf = pd.read_parquet('../data_analysis/nsf_analysis.parquet', engine='fastparquet')

In [ ]:
nsf_raw = pd.read_csv('../data_raw/grant_witness_nsf_terminations.csv')
nih_raw = pd.read_csv('../data_raw/grant_witness_nih_terminations.csv')

### 1.2 Column Grouping 

To understand the meaning of the columns better, we put them into more high-level groups and organize the data frames' columns into a corresponding multi-index. 

In [ ]:
nsf.columns

Index(['application_id', 'core_project_num', 'administering_ic',
       'administering_ic_name', 'study_section', 'study_section_name',
       'pi_names', 'org_name', 'org_state', 'congressional_district',
       'org_dept', 'organization_type', 'project_title', 'project_terms',
       'public_health_relevance', 'nih_spending_categorization',
       'project_abstract', 'fiscal_year', 'project_start', 'project_end',
       'targeted_start_date', 'targeted_end_date', 'frozen_date',
       'unfrozen_date', 'termination_date', 'reinstatement_indicator',
       'reinstated_est_date', 'total_cost', 'total_estimated_outlays',
       'total_estimated_remaining', 'last_payment_date', 'status',
       'ever_frozen', 'cancellation_source', 'flagged_words', 'us_rep',
       'usaspending_url', 'reporter_url'],
      dtype='object')

In [185]:
nsf.columns

Index(['grant_id', 'agcy_id', 'pi_names', 'pi_emails', 'title', 'abstract',
       'start_date', 'end_date', 'total_budget', 'amount_awarded', 'org_name',
       'org_state', 'was_cut', 'terminated', 'suspended', 'termination_date',
       'reinstated', 'reinstatement_date', 'nsf_url', 'usaspending_url',
       'award_type', 'division', 'directorate', 'nsf_program_name',
       'usasp_outlaid', 'estimated_remaining',
       'post_termination_deobligation'],
      dtype='object')

In [ ]:
##################
#### NIH Data ####
##################

nih_column_mapping = {
    # Identification: Unique keys and external database links
    'application_id': 'id', 
    'core_project_num': 'id', 
    'reporter_url': 'id', 
    'usaspending_url': 'id',
    
    # Context: The administrative, institutional, and political ecosystem
    'administering_ic': 'context', 
    'administering_ic_name': 'context',
    'study_section': 'context', 
    'study_section_name': 'context',
    'pi_names': 'context', 
    'org_name': 'context', 
    'org_dept': 'context', 
    'organization_type': 'context', 
    'org_state': 'context', 
    'congressional_district': 'context', 
    'us_rep': 'context',
    
    # Content: Scientific substance and research focus
    'project_title': 'content', 
    'project_abstract': 'content', 
    'project_terms': 'content', 
    'public_health_relevance': 'content',
    'nih_spending_categorization': 'content', 
    'flagged_words': 'content',
    
    # Time: Temporal data regarding the grant lifecycle
    'fiscal_year': 'time', 
    'project_start': 'time', 
    'project_end': 'time',
    'targeted_start_date': 'time', 
    'targeted_end_date': 'time', 
    'frozen_date': 'time', 
    'unfrozen_date': 'time', 
    'termination_date': 'time',
    
    # Finance: Budgetary figures and spending metrics
    'total_cost': 'finance', 
    'total_estimated_outlays': 'finance', 
    'total_estimated_remaining': 'finance', 
    'last_payment_date': 'finance',
    
    # Status: Indicators of termination and reinstatement
    'status': 'status', 
    'ever_frozen': 'status', 
    'reinstatement_indicator': 'status', 
    'reinstated_est_date': 'status', 
    'cancellation_source': 'status'
}

nih.columns = pd.MultiIndex.from_tuples(
    [(nih_column_mapping.get(col, 'Other'), col) for col in nih.columns],
    names=['category', 'variable']
)

In [ ]:
##################
#### NSF Data ####
##################

nsf_column_mapping = {
    # Identification: IDs and technical links
    'grant_id': 'id',
    'agcy_id': 'id',
    'nsf_url': 'id',
    'usaspending_url': 'id',
    
    # Context: Institutional and organizational structure
    'pi_names': 'context',
    'pi_emails': 'context',
    'org_name': 'context',
    'org_state': 'context',
    'division': 'context', 
    'directorate': 'context',
    'nsf_program_name': 'context',
    'award_type': 'context',   
    
    # Content: Scientific substance
    'title': 'content',
    'abstract': 'content',
    
    # Time: Temporal grant lifecycle
    'start_date': 'time',
    'end_date': 'time',
    'termination_date': 'time',
    'reinstatement_date': 'time',
    
    # Finance: Budgetary metrics
    'total_budget': 'finance',
    'amount_awarded': 'finance',
    'usasp_outlaid': 'finance',
    'estimated_remaining': 'finance',
    'post_termination_deobligation': 'finance',
    
    # Status: Termination indicators
    'was_cut': 'status',
    'terminated': 'status',
    'suspended': 'status',
    'reinstated': 'status'
}

nsf.columns = pd.MultiIndex.from_tuples(
    [(nsf_column_mapping.get(col, 'Other'), col) for col in nsf.columns],
    names=['Category', 'Variable']
)

### 1.3 Data Quality Checks

#### Reliability of total budget information

In [262]:
nih[('finance', 'check_sum')] = nih[('finance', 'total_estimated_outlays')] + nih[('finance', 'total_estimated_remaining')]

cost = nih[('finance', 'total_cost')]
check = nih[('finance', 'check_sum')]

total_cost_false = len(nih[(cost > 1.1 * check) | (cost < 0.9 * check)]['finance'])
total_cost_correct = len(nih[(cost < 1.1 * check) & (cost > 0.9 * check)]['finance'])

grant_witness_mask = nih[('status', 'status')].isna() == False
missing_fin_inf_mask = (nih[('finance', 'total_estimated_outlays')].isna() == True) | (nih[('finance', 'total_estimated_remaining')].isna() == True)
missing_total_cost_mask = cost.isna() == True

print(f"Total entries NIH data: {len(nih)}")
print(f"NIH entries from grant-witness.us: {len(nih[grant_witness_mask])}")
print("----------------------------------")
print(f"NIH entries without total_cost info: {sum(cost.isna())}")
print(f"NIH grant-witness.us entries without total_cost info: {len(nih[grant_witness_mask & (missing_total_cost_mask)])}")
print(f"NIH grant-witness.us entries without further financial information: {len(nih[grant_witness_mask & missing_fin_inf_mask])}")
print("----------------------------------")
print(f"Entries where total_cost equals sum of grant-witness.us financial info: {total_cost_correct}")
print(f"Entries where total_cost does not equal sum of grant-witness.us financial info: {total_cost_false}")
print("----------------------------------")
print("----------------------------------")

pd.DataFrame({
    'measure': ['entries with total_cost info', 'grant-witness entries with correct total_cost info'],
    'value': [1 - (sum(cost.isna()) / len(nih)), (total_cost_correct / len(nih[grant_witness_mask]))]
})

Total entries NIH data: 230456
NIH entries from grant-witness.us: 5814
----------------------------------
NIH entries without total_cost info: 9920
NIH grant-witness.us entries without total_cost info: 112
NIH grant-witness.us entries without further financial information: 55
----------------------------------
Entries where total_cost equals sum of grant-witness.us financial info: 551
Entries where total_cost does not equal sum of grant-witness.us financial info: 5096
----------------------------------
----------------------------------


,measure,value
0,entries with total_cost info,0.956955
1,grant-witness entries with correct total_cost ...,0.094771


In [263]:
nsf[('finance', 'check_sum')] = nsf[('finance', 'usasp_outlaid')] + nsf[('finance', 'estimated_remaining')]

cost = nsf[('finance', 'total_budget')]
check = nsf[('finance', 'check_sum')]

total_cost_false = len(nsf[(cost > 1.1 * check) | (cost < 0.9 * check)]['finance'])
total_cost_correct = len(nsf[(cost < 1.1 * check) & (cost > 0.9 * check)]['finance'])

grant_witness_mask = nsf[('status', 'terminated')].isna() == False
missing_fin_inf_mask = (nsf[('finance', 'usasp_outlaid')].isna() == True) | (nsf[('finance', 'estimated_remaining')].isna() == True)
missing_total_cost_mask = cost.isna() == True

print(f"Total entries NIH data: {len(nsf)}")
print(f"NIH entries from grant-witness.us: {len(nsf[grant_witness_mask])}")
print("----------------------------------")
print(f"NIH entries without total_budget info: {sum(cost.isna())}")
print(f"NIH grant-witness.us entries without total_budget info: {len(nsf[grant_witness_mask & (missing_total_cost_mask)])}")
print(f"NIH grant-witness.us entries without further financial information: {len(nsf[grant_witness_mask & missing_fin_inf_mask])}")
print("----------------------------------")
print(f"Entries where total_budget equals sum of grant-witness.us financial info: {total_cost_correct}")
print(f"Entries where total_budget does not equal sum of grant-witness.us financial info: {total_cost_false}")
print("----------------------------------")
print("----------------------------------")

pd.DataFrame({
    'measure': ['entries with total_budget info', 'grant-witness entries with correct total_budget info'],
    'value': [1 - (sum(cost.isna()) / len(nsf)), (total_cost_correct / len(nsf[grant_witness_mask]))]
})

Total entries NIH data: 130204
NIH entries from grant-witness.us: 1890
----------------------------------
NIH entries without total_budget info: 2
NIH grant-witness.us entries without total_budget info: 0
NIH grant-witness.us entries without further financial information: 90
----------------------------------
Entries where total_budget equals sum of grant-witness.us financial info: 1706
Entries where total_budget does not equal sum of grant-witness.us financial info: 94
----------------------------------
----------------------------------


,measure,value
0,entries with total_budget info,0.999985
1,grant-witness entries with correct total_budge...,0.902646


#### Content information

In [283]:
def check_content_missingness(data):
    try:
        titles = data[('content', 'project_title')]
        abstracts = data[('content', 'project_abstract')]
    except:
        titles = data[('content', 'title')]
        abstracts = data[('content', 'abstract')]

    return pd.DataFrame({
        'measure': ['entries w/ titles', 'entries w/ abstracts', 'w/ title w/o abstract'],
        'value': [
                    len(titles[titles.isna() == False]) / len(data), 
                    len(abstracts[abstracts.isna() == False]) / len(data),
                    len(data[(titles.isna() == False) & (abstracts.isna() == True)]) / len(data)
                ]
    })

In [284]:
check_content_missingness(nih)

,measure,value
0,entries w/ titles,1.000000
1,entries w/ abstracts,0.968571
2,w/ title w/o abstract,0.031429


In [285]:
check_content_missingness(nsf)

,measure,value
0,entries w/ titles,1.000000
1,entries w/ abstracts,0.995177
2,w/ title w/o abstract,0.004823
